Fuente: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [1]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [2]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [3]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [4]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation


In [5]:
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))



In [6]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs


In [7]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

In [8]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
                    print("error: {}".format(error))
				errors.append(error)
		else:
            print("In last layer...")
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
                print("error: {}".format(neuron['output'] - expected[j]))
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [9]:
# Update network weights with error
def update_weights(network, row, l_rate):
    print("Updating weights")
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

In [10]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

In [11]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [12]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))


In [13]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)


In [14]:
# load and prepare data
filename = 'data/wheat-seeds.csv'
dataset = load_csv(filename)
dataset

[['15.26', '14.84', '0.871', '5.763', '3.312', '2.221', '5.22', '1'],
 ['14.88', '14.57', '0.8811', '5.554', '3.333', '1.018', '4.956', '1'],
 ['14.29', '14.09', '0.905', '5.291', '3.337', '2.699', '4.825', '1'],
 ['13.84', '13.94', '0.8955', '5.324', '3.379', '2.259', '4.805', '1'],
 ['16.14', '14.99', '0.9034', '5.658', '3.562', '1.355', '5.175', '1'],
 ['14.38', '14.21', '0.8951', '5.386', '3.312', '2.462', '4.956', '1'],
 ['14.69', '14.49', '0.8799', '5.563', '3.259', '3.586', '5.219', '1'],
 ['14.11', '14.1', '0.8911', '5.42', '3.302', '2.70', '5.00', '1'],
 ['16.63', '15.46', '0.8747', '6.053', '3.465', '2.04', '5.877', '1'],
 ['16.44', '15.25', '0.888', '5.884', '3.505', '1.969', '5.533', '1'],
 ['15.26', '14.85', '0.8696', '5.714', '3.242', '4.543', '5.314', '1'],
 ['14.03', '14.16', '0.8796', '5.438', '3.201', '1.717', '5.001', '1'],
 ['13.89', '14.02', '0.888', '5.439', '3.199', '3.986', '4.738', '1'],
 ['13.78', '14.06', '0.8759', '5.479', '3.156', '3.136', '4.872', '1'],
 [

In [15]:
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
dataset

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, '1'],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, '1'],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, '1'],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, '1'],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, '1'],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, '1'],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, '1'],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, '1'],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, '1'],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, '1'],
 [15.26, 14.85, 0.8696, 5.714, 3.242, 4.543, 5.314, '1'],
 [14.03, 14.16, 0.8796, 5.438, 3.201, 1.717, 5.001, '1'],
 [13.89, 14.02, 0.888, 5.439, 3.199, 3.986, 4.738, '1'],
 [13.78, 14.06, 0.8759, 5.479, 3.156, 3.136, 4.872, '1'],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, '1'],
 [14.59, 14.28, 0.8993, 5.351, 3.333, 4.185, 4.781, '1'],
 [13.99, 13.83, 0.9183, 5.119, 3.383, 5.234, 4.781, '1'],
 [15.69, 14.75, 0.9058, 5.

In [16]:
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
dataset

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 1],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 1],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 1],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, 1],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, 1],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, 1],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, 1],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, 1],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, 1],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, 1],
 [15.26, 14.85, 0.8696, 5.714, 3.242, 4.543, 5.314, 1],
 [14.03, 14.16, 0.8796, 5.438, 3.201, 1.717, 5.001, 1],
 [13.89, 14.02, 0.888, 5.439, 3.199, 3.986, 4.738, 1],
 [13.78, 14.06, 0.8759, 5.479, 3.156, 3.136, 4.872, 1],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, 1],
 [14.59, 14.28, 0.8993, 5.351, 3.333, 4.185, 4.781, 1],
 [13.99, 13.83, 0.9183, 5.119, 3.383, 5.234, 4.781, 1],
 [15.69, 14.75, 0.9058, 5.527, 3.514, 1.599, 5.046, 1],
 [14

In [17]:
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
dataset

[[0.4409820585457979,
  0.5020661157024793,
  0.570780399274047,
  0.48648648648648646,
  0.48610121168923714,
  0.18930164220052273,
  0.3451501723289019,
  1],
 [0.40509915014164316,
  0.44628099173553726,
  0.6624319419237747,
  0.3688063063063065,
  0.5010691375623664,
  0.03288301759221938,
  0.21516494337764666,
  1],
 [0.3493862134088762,
  0.3471074380165289,
  0.8793103448275864,
  0.22072072072072094,
  0.50392017106201,
  0.25145301590191005,
  0.15066469719350079,
  1],
 [0.3068932955618508,
  0.31611570247933873,
  0.7931034482758617,
  0.23930180180180172,
  0.5338560228082679,
  0.19424254638598865,
  0.14081733136386,
  1],
 [0.5240793201133145,
  0.5330578512396694,
  0.8647912885662429,
  0.4273648648648651,
  0.6642908054169634,
  0.076701036289641,
  0.32299359921221066,
  1],
 [0.3578847969782815,
  0.37190082644628114,
  0.7894736842105262,
  0.2742117117117118,
  0.48610121168923714,
  0.22063737663992516,
  0.21516494337764666,
  1],
 [0.3871576959395656,
  0.42

In [19]:
# Test Backprop on Seeds dataset
seed(0)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [92.85714285714286, 92.85714285714286, 97.61904761904762, 90.47619047619048, 100.0]
Mean Accuracy: 94.762%
